# **Fine-tuning XLSR-Wav2Vec2 for Egyptian speech recognition**

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.  Soon after the superior performance of Wav2Vec2 was demonstrated on the English ASR dataset LibriSpeech, *Facebook AI* presented XLSR-Wav2Vec2 (click [here](https://arxiv.org/abs/2006.13979)). XLSR stands for *cross-lingual  speech representations* and refers to XLSR-Wav2Vec2's ability to learn speech representations that are useful across multiple languages.

Similar to Wav2Vec2, XLSR-Wav2Vec2 learns powerful speech representations from hundreds of thousands of hours of speech in more than 50 languages of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/xlsr_wav2vec2.png)

The authors show for the first time that massively pretraining an ASR model on cross-lingual unlabeled speech data, followed by language-specific fine-tuning on very little labeled data achieves state-of-the-art results. See Table 1-5 of the official [paper](https://arxiv.org/pdf/2006.13979.pdf).

In this notebook, we will give an in-detail explanation of how XLSR-Wav2Vec2's pretrained checkpoint can be fine-tuned on a low-resource ASR dataset of any language. Note that in this notebook, we will fine-tune XLSR-Wav2Vec2 without making use of a language model. It is much simpler and more efficient to use XLSR-Wav2Vec2 without a language model, but better results can be achieved by including a language model. 

For demonstration purposes, we fine-tune the [wav2vec2-large-xlsr-53](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) on the low resource Turkish ASR dataset of [Common Voice](https://huggingface.co/datasets/common_voice) that contains just ~6h of validated training data.

XLSR-Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. 

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `torchaudio` and `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install datasets==2.1.0
!pip install transformers==4.18.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
import librosa
print(librosa.__version__)

0.8.1





---

${}^1$ In the [paper](https://arxiv.org/pdf/2006.13979.pdf), the model was evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text. 

In 🤗 Transformers, the XLSR-Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned XLSR-Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *'BERT'* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on XLSR-Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Common Voice and define a vocabulary based on the dataset's transcriptions.

First, let's go to [Common Voice](https://commonvoice.mozilla.org/en/datasets) and pick a language to fine-tune XLSR-Wav2Vec2 on. For this notebook, we will use Turkish. 

For each language-specific dataset, you can find a language code corresponding to your chosen language. On [Common Voice](https://commonvoice.mozilla.org/en/datasets), look for the field "Version". The language code then corresponds to the prefix before the underscore. For Turkish, *e.g.* the language code is `"tr"`.

Great, now we can use 🤗 Datasets' simple API to download the data. The dataset name will be `"common_voice"`, the config name corresponds to the language code - `"tr"` in our case.

Common Voice has many different splits including `invalidated`, which refers to data that was not rated as "clean enough" to be considered useful. In this notebook, we will only make use of the splits `"train"`, `"validation"` and `"test"`. 

Because the Turkish dataset is so small, we will merge both the validation and training data into a training dataset and simply use the test data for validation.

**Loading Our dataset**

In [ ]:

import pandas as pd 
import re
import unicodedata
!mkdir audio
!cp '/content/drive/MyDrive/Final_speech/wav_files.rar' '/content/audio/'
!cp '/content/drive/MyDrive/Final_speech/path.txt' '/content/audio/'
!cp '/content/drive/MyDrive/Final_speech/transcription2.txt' '/content/audio/'
!cp '/content/drive/MyDrive/Final_speech/one_channel.csv' '/content/'
!cp '/content/drive/MyDrive/Final_speech/one_channel.rar' '/content/audio/'
!unrar  x /content/audio/wav_files.rar 
#!unrar  x /content/audio/one_channel.rar



UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from /content/audio/wav_files.rar

Creating    wav_files                                                 OK
Extracting  wav_files/0.wav                                                0%  OK 
Extracting  wav_files/1.wav                                                0%  OK 
Extracting  wav_files/10.wav                                               0%  OK 
Extracting  wav_files/100.wav                                              0%  OK 
Extracting  wav_files/1000.wav                                             0%  OK 
Extracting  wav_files/1001.wav                                             0%  OK 
Extracting  wav_files/1002.wav                                             0%  OK 
Extracting  wav_files/1003.wav                                             0%  OK 
Extracting  wav_files/1004.wav                                       

In [ ]:
'''
!mkdir audio
!cp '/content/drive/MyDrive/Final_speech/path.txt' '/content/audio/'
!cp '/content/drive/MyDrive/Final_speech/transcription2.txt' '/content/audio/'
'''

"\n!mkdir audio\n!cp '/content/drive/MyDrive/Final_speech/path.txt' '/content/audio/'\n!cp '/content/drive/MyDrive/Final_speech/transcription2.txt' '/content/audio/'\n"

In [ ]:
import pandas as pd 
import re
import unicodedata

In [ ]:
import pandas as pd 
df5=pd.read_csv("/content/drive/MyDrive/Final_speech/cleaned_20_1.csv")
df5=df5.drop(columns=['Unnamed: 0'])
df5.head()

,sentence,audio
0,اهلا بحضرتك و أنا سعيدة جدا اني موجودة معكم ال...,/content/wav_files/0.wav
1,جميل جدا قبل اي حاجة حابة اتكلم معاك عن الدراسة.,/content/wav_files/1.wav
2,و اهميتها.,/content/wav_files/2.wav
3,وبعدين ندخل آه في التعليم عن بعد.,/content/wav_files/3.wav
4,اللي هو ظهر بفترة.,/content/wav_files/4.wav


In [ ]:
!pip install audiomentations 

In [ ]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, Normalize
import numpy as np
import librosa

augment = Compose([
     Normalize(),
     AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
     #TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
     #PitchShift(min_semitones=-4, max_semitones=4, p=0.5)
   
])



In [ ]:
'''
##clean the data from records that contain english or numbers(numbers is not transcripted in the the transcription file so they will be removed)
# found 796 record that contain english or numbers and will be removed from the set
import pandas as pd 
import re
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\'ُ\%\‘\”\�\+\؟\[\]\،\\*\'\\&\\ufeff\'ً\’\ـ\'ّ]'

pattern = re.compile("[A-Za-z0-9]+")
df1 = pd.DataFrame(columns=['sentence','audio'])
#df=pd.read_csv( "/content/drive/MyDrive/Final_speech/cleaned_audios.csv")
sente=df5['sentence']
path=df5['audio']
c=0
txt=[]
paths=[]
for i,p in zip(sente,path):
    i=re.sub(chars_to_ignore_regex, '',i)
    txt.append(i.strip()+".")
    paths.append(p)
    

df1['sentence']=txt
df1['audio']=paths
df1.to_csv ('/content/drive/MyDrive/Final_speech/cleaned_20_1.csv')
df1.head()
'''

'\n##clean the data from records that contain english or numbers(numbers is not transcripted in the the transcription file so they will be removed)\n# found 796 record that contain english or numbers and will be removed from the set\nimport pandas as pd \nimport re\nimport re\nchars_to_ignore_regex = \'[\\,\\?\\.\\!\\-\\;\\:"\\“\'ُ\\%\\‘\\”\\�\\+\\؟\\[\\]\\،\\*\'\\&\\ufeff\'ً\\’\\ـ\'ّ]\'\n\npattern = re.compile("[A-Za-z0-9]+")\ndf1 = pd.DataFrame(columns=[\'sentence\',\'audio\'])\n#df=pd.read_csv( "/content/drive/MyDrive/Final_speech/cleaned_audios.csv")\nsente=df5[\'sentence\']\npath=df5[\'audio\']\nc=0\ntxt=[]\npaths=[]\nfor i,p in zip(sente,path):\n    i=re.sub(chars_to_ignore_regex, \'\',i)\n    txt.append(i.strip()+".")\n    paths.append(p)\n    \n\ndf1[\'sentence\']=txt\ndf1[\'audio\']=paths\ndf1.to_csv (\'/content/drive/MyDrive/Final_speech/cleaned_20_1.csv\')\ndf1.head()\n'

In [ ]:
#!pip install soundfile

In [ ]:
'''
import soundfile as sf
sente=df5['sentence']
path=df5['audio']
c=0
txt=[]
paths=[]
for i,p in zip(sente,path):
  ob = sf.SoundFile(p)
  print('Sample rate: {} Channels: {} Subtype: {},path: {}'.format(ob.samplerate,ob.channels,ob.subtype,p))
'''

"\nimport soundfile as sf\nsente=df5['sentence']\npath=df5['audio']\nc=0\ntxt=[]\npaths=[]\nfor i,p in zip(sente,path):\n  ob = sf.SoundFile(p)\n  print('Sample rate: {} Channels: {} Subtype: {},path: {}'.format(ob.samplerate,ob.channels,ob.subtype,p))\n"

In [ ]:
'''
##clean the data from records that contain english or numbers(numbers is not transcripted in the the transcription file so they will be removed)
# found 796 record that contain english or numbers and will be removed from the set
import pandas as pd 
import re
pattern = re.compile("[A-Za-z0-9]+")
df1 = pd.DataFrame(columns=['sentence','audio'])
df=pd.read_csv( "/content/drive/MyDrive/Final_speech/cleaned_audios.csv")
sente=df['sentence']
path=df['audio']
c=0
txt=[]
paths=[]
for i,p in zip(sente,path):
 if re.search("[A-Za-z0-9]+", i) is None :
   txt.append(i)
   paths.append(p)
   c+=1
#  else:
#     print(i)
print(c)

df1['sentence']=txt
df1['audio']=paths
df1.to_csv ('/content/drive/MyDrive/Final_speech/cleaned_18_1.csv')
df1.head()
'''




'\n##clean the data from records that contain english or numbers(numbers is not transcripted in the the transcription file so they will be removed)\n# found 796 record that contain english or numbers and will be removed from the set\nimport pandas as pd \nimport re\npattern = re.compile("[A-Za-z0-9]+")\ndf1 = pd.DataFrame(columns=[\'sentence\',\'audio\'])\ndf=pd.read_csv( "/content/drive/MyDrive/Final_speech/cleaned_audios.csv")\nsente=df[\'sentence\']\npath=df[\'audio\']\nc=0\ntxt=[]\npaths=[]\nfor i,p in zip(sente,path):\n if re.search("[A-Za-z0-9]+", i) is None :\n   txt.append(i)\n   paths.append(p)\n   c+=1\n#  else:\n#     print(i)\nprint(c)\n\ndf1[\'sentence\']=txt\ndf1[\'audio\']=paths\ndf1.to_csv (\'/content/drive/MyDrive/Final_speech/cleaned_18_1.csv\')\ndf1.head()\n'

In [ ]:
'''
new_rec=pd.read_csv( "/content/drive/MyDrive/Final_speech/one_channel.csv")
new_rec=new_rec.drop(columns=['Unnamed: 0'])
new_rec
'''

'\nnew_rec=pd.read_csv( "/content/drive/MyDrive/Final_speech/one_channel.csv")\nnew_rec=new_rec.drop(columns=[\'Unnamed: 0\'])\nnew_rec\n'

In [ ]:
## some symbols exists in the transcription that needs to be cleaned

symbols=["[*]","[UNK]","[+]","[LAUGHTER]","[SONANT]","[MUSIC]","[SYSTEM]", 
                  "[ENS]" ,"[NPS]" ,"[PII]"]

In [ ]:

import pandas as pd
paths=[]
txt=[]
count=0
df = pd.DataFrame(columns=['sentence','audio'])
with open('/content/audio/path.txt','r',encoding='utf-8') as path:
  with open('/content/audio/transcription2.txt','r',encoding='utf-8') as t:
    p_lines=path.readlines()
    t_lines=t.readlines()
    for j in range(len(p_lines)):
      flag=0
      p=p_lines[j].strip()
      tr=t_lines[j].strip()
      # reomve tashkeel from transcript
      tr = ''.join([t for t in tr if t not in [chr(x) for x in range(0x0600, 0x06ff) if unicodedata.category(chr(x)) == "Mn"]])

      for symb in symbols:
        if tr==symb:
          #print(symb)
          flag=1
          count+=1
          break
      if flag ==0 :
        txt.append(tr)
        paths.append(p)
        
    #df['path']=paths
    df['sentence']=txt
    df['audio']=paths


In [ ]:
## Merge old and new records dataframe
'''
df3=pd.concat([df, new_rec], axis=0)
df3
'''

'\ndf3=pd.concat([df, new_rec], axis=0)\ndf3\n'

In [ ]:
'''
import librosa
import soundfile as sf
import torch
paths=df3['audio']
trans=df3['sentence']
p=[]
t=[]
for path,tr in zip(paths,trans):
  try :
    speech, rate = sf.read(path)
    speech = librosa.resample(speech, rate, 16000)
    p.append(path)
    t.append(tr)
  except:
    print("error happened in ",path)
'''

'\nimport librosa\nimport soundfile as sf\nimport torch\npaths=df3[\'audio\']\ntrans=df3[\'sentence\']\np=[]\nt=[]\nfor path,tr in zip(paths,trans):\n  try :\n    speech, rate = sf.read(path)\n    speech = librosa.resample(speech, rate, 16000)\n    p.append(path)\n    t.append(tr)\n  except:\n    print("error happened in ",path)\n'

In [ ]:
'''
import librosa
import soundfile as sf
import torch
paths=df4['audio']
trans=df4['sentence']
import re
p=[]
t=[]
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\+\؟\[\]\،\\*\'\\&\\ufeff\'ً\’\ـ]'
p=[]
t=[]
for path,tr in zip(paths,trans):
  try :
    tr=tr.strip()
    tr = re.sub(chars_to_ignore_regex, '',tr).lower() + " "
   
    tr = re.sub('[a-z]','',tr)    # remove any non arabic char exist in transcription
    tr = re.sub("[إأٱآا]", "ا", tr)
    tr = re.sub('[0-9]', '', tr)   #replace all the "ا" wih "ا"
    if len(tr.strip())==0:
      
      print("found",path)
      #break
    else:
       p.append(path)
       t.append(tr)


    

    # speech, rate = sf.read(path)
    # speech = librosa.resample(speech, rate, 16000)
    # p.append(path)
    # t.append(tr)
  except:
    print("error happened in ",path)

df5 = pd.DataFrame(columns=['sentence','audio'])
df5['audio']=p
df5['sentence']=t
df5.to_csv ('/content/drive/MyDrive/Final_speech/cleaned2.csv')
df5.head()
'''

'\nimport librosa\nimport soundfile as sf\nimport torch\npaths=df4[\'audio\']\ntrans=df4[\'sentence\']\nimport re\np=[]\nt=[]\nchars_to_ignore_regex = \'[\\,\\?\\.\\!\\-\\;\\:"\\“\\%\\‘\\”\\�\\+\\؟\\[\\]\\،\\*\'\\&\\ufeff\'ً\\’\\ـ]\'\np=[]\nt=[]\nfor path,tr in zip(paths,trans):\n  try :\n    tr=tr.strip()\n    tr = re.sub(chars_to_ignore_regex, \'\',tr).lower() + " "\n   \n    tr = re.sub(\'[a-z]\',\'\',tr)    # remove any non arabic char exist in transcription\n    tr = re.sub("[إأٱآا]", "ا", tr)\n    tr = re.sub(\'[0-9]\', \'\', tr)   #replace all the "ا" wih "ا"\n    if len(tr.strip())==0:\n      \n      print("found",path)\n      #break\n    else:\n       p.append(path)\n       t.append(tr)\n\n\n    \n\n    # speech, rate = sf.read(path)\n    # speech = librosa.resample(speech, rate, 16000)\n    # p.append(path)\n    # t.append(tr)\n  except:\n    print("error happened in ",path)\n\ndf5 = pd.DataFrame(columns=[\'sentence\',\'audio\'])\ndf5[\'audio\']=p\ndf5[\'sentence\']=t\ndf5.to

In [ ]:
'''
df4 = pd.DataFrame(columns=['sentence','audio'])
df4['audio']=p
df4['sentence']=t
df4.head()4
'''

"\ndf4 = pd.DataFrame(columns=['sentence','audio'])\ndf4['audio']=p\ndf4['sentence']=t\ndf4.head()4\n"

In [ ]:
#df4.to_csv ('/content/drive/MyDrive/Final_speech/cleaned_audios.csv')

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)


In [ ]:
##splitting the dataset into train and test
train_test=dataset.train_test_split(test_size=0.05)
train_dataset=train_test['train']
test_dataset=train_test['test']
test_dataset1=train_test['test']

## train validation
# train_valid=train_dataset.train_test_split(test_size=0.05)
# train_dataset=train_valid['train']
# valid_dataset=train_valid['test']

In [ ]:
import datasets
train_dataset=train_dataset.cast_column('audio', datasets.Audio(sampling_rate=16000))
test_dataset=test_dataset.cast_column('audio', datasets.Audio(sampling_rate=16000))
#valid_dataset=valid_dataset.cast_column('audio', datasets.Audio(sampling_rate=16000))


Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_dataset.remove_columns(["audio"]), num_examples=20)

,sentence
0,آه في كثير أسر ما يعني ما بتهتمش
1,أخرى، أتطرق معك لهذا الموضوع
2,سمعاكي اتفضلي
3,إني السوشيال ميديا
4,الرطوبة بتاعته، التيارات بتاعت البحر، البحر نفسه أساسا
5,والمفعول لأجله
6,عندك فكرة عنها
7,انه احنا نتجنب الرياضة
8,احن وصلنا لمرحلة
9,الشهادة مثلا اللي أنت بتاخديها في آخر السنة بتكون مادة الرياضة تحتسب معاها


Alright! The transcriptions look fairly clean. Having translated the transcribed sentences, it seems that the language corresponds more to written-out text than noisy dialogue. This makes sense considering that [Common Voice](https://huggingface.co/datasets/common_voice) is a crowd-sourced read speech corpus.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\'ُ\%\‘\”\�\+\؟\[\]\،\\*\'\\&\\ufeff\'ً\’\ـ\'ّ]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    #batch["sentence"] = re.sub('[a-z]','',batch["sentence"])    # remove any non arabic char exist in transcription
    batch["sentence"] = re.sub("[إأٱآاء]", "ا", batch["sentence"])
    batch["sentence"] = re.sub("[ي]", "ى", batch["sentence"])
    batch["sentence"] = re.sub("[ة]", "ه", batch["sentence"])
    batch["sentence"] = re.sub('[0-9]', '', batch["sentence"])   #replace all the "ا" wih "ا"
    #batch["sentence"] = re.sub("[]", "ا", batch["sentence"])
    batch["audio"]['array']=augment(samples=batch["audio"]['array'], sample_rate=16000)

    




    return batch

In [ ]:
test_dataset['audio'][0]

{'path': '/content/wav_files/199.wav',
 'array': array([ 0.00015259,  0.00015259,  0.00012207, ...,  0.00119019,
        -0.00039673, -0.00164795], dtype=float32),
 'sampling_rate': 16000}

In [ ]:
train_dataset=train_dataset.map(remove_special_characters)
test_dataset=test_dataset.map(remove_special_characters)
#valid_dataset=valid_dataset.map(remove_special_characters)


  0%|          | 0/2901 [00:00<?, ?ex/s]

  0%|          | 0/153 [00:00<?, ?ex/s]

In [ ]:
show_random_elements(test_dataset.remove_columns(["audio"]))

,sentence
0,اه شوفى زى ما انا قولتلك ان انا بتكلم على اسلوب حىاه
1,او كوب من الزىادى او كوب من الحلىب
2,ىعنى ولو
3,من ناحىه انه
4,والهواتف المحموله
5,اه احنا حابىن نتكلم معاك النهار ده عن موضوع هو فى غاىه الاهمىه
6,من اهم الفنون اه و ان هو بىكون
7,اه عاىزىن نتكلم عن اى المفروض ىكون موجود فىها وبعد كدا تكلمىنى عن وجبه الغداا
8,و انى انا ازاى بدخل له اكل معىن
9,قاره اسىا معروف انها برد


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here. 
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. 
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
test_vocab = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)
train_vocab = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
#valid_vocab = valid_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [ ]:
vocab_list = list(set(train_vocab["vocab"][0]) | set(test_vocab["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'س': 0,
 'ع': 1,
 'ى': 2,
 'ج': 3,
 'ش': 4,
 'ك': 5,
 'ف': 6,
 'ق': 7,
 'ذ': 8,
 'ص': 9,
 'ظ': 10,
 'ت': 11,
 'ؤ': 12,
 'غ': 13,
 'ط': 14,
 'ل': 15,
 'ض': 16,
 'ب': 17,
 'خ': 18,
 'م': 19,
 'و': 20,
 'ث': 21,
 'ا': 22,
 'ر': 23,
 'ن': 24,
 ' ': 25,
 'ئ': 26,
 'ح': 27,
 'ه': 28,
 'ز': 29,
 'د': 30}

**The above vocab seems very nice**

Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `" "` and `'`. Note that we did not exclude those special characters because: 

- The model has to learn to predict when a word is finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- From the transcriptions above it seems that words that include an apostrophe, such as `maktouf'un` do exist in Turkish, so I decided to keep the apostrophe in the dataset. This might be a wrong assumption though.

One should always keep in mind that the data-preprocessing is a very important step before training your model. E.g., we don't want our model to differentiate between `a` and `A` just because we forgot to normalize the data. The difference between `a` and `A` does not depend on the "sound" of the letter at all, but more on grammatical rules - *e.g.* use a capitalized letter at the beginning of the sentence. So it is sensible to remove the difference between capitalized and non-capitalized letters so that the model has an easier time learning to transcribe speech. 

It is always advantageous to get help from a native speaker of the language you would like to transcribe to verify whether the assumptions you made are sensible, *e.g.* I should have made sure that keeping `'`, but removing other special characters is a sensible choice for Turkish. 

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Common Voice's training set. 

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

33

**Now,vocab contains 36 char which is very reosnable**

In [ ]:
vocab_dict

{'س': 0,
 'ع': 1,
 'ى': 2,
 'ج': 3,
 'ش': 4,
 'ك': 5,
 'ف': 6,
 'ق': 7,
 'ذ': 8,
 'ص': 9,
 'ظ': 10,
 'ت': 11,
 'ؤ': 12,
 'غ': 13,
 'ط': 14,
 'ل': 15,
 'ض': 16,
 'ب': 17,
 'خ': 18,
 'م': 19,
 'و': 20,
 'ث': 21,
 'ا': 22,
 'ر': 23,
 'ن': 24,
 'ئ': 26,
 'ح': 27,
 'ه': 28,
 'ز': 29,
 'د': 30,
 '|': 25,
 '[UNK]': 31,
 '[PAD]': 32}

Cool, now our vocabulary is complete and consists of 39 tokens, which means that the linear layer that we will add on top of the pretrained XLSR-Wav2Vec2 checkpoint will have an output dimension of 39.

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Next, we will create the feature extractor.

### Create XLSR-Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus, 
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

XLSR-Wav2Vec2 was pretrained on the audio data of [Babel](https://huggingface.co/datasets/librispeech_asr), 
[Multilingual LibriSpeech (MLS)](https://ai.facebook.com/blog/a-new-open-data-set-for-multilingual-speech-research/), and [Common Voice](https://huggingface.co/datasets/common_voice). Most of those datasets were sampled at 16kHz, so that Common Voice, sampled at 48kHz, has to be downsampled to 16kHz for training. Therefore, we will have to downsample our fine-tuning data to 16kHz in the following.



A XLSR-Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, XLSR-Wav2Vec2 models should **always** make use of the `attention_mask`.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

Great, XLSR-Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of XLSR-Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

If one wants to re-use the just created processor and the fine-tuned model of this notebook, one can mount his/her google drive to the notebook and save all relevant files there. To do so, please uncomment the following lines. 

We will give the fine-tuned model the name `"wav2vec2-large-xlsr-turkish-demo"`.

In [ ]:
processor.save_pretrained("/content/drive/MyDrive/22new_wav2vec2-large-xlsr-egyptian-demo")

In [ ]:
##To load 
from transformers import Wav2Vec2Processor
#processor=Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/Final_speech/gabrprocessor ")

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just the transcriptio. In addition to `sentence`, our datasets include two more column names `path` and `audio`. `path` states the absolute path of the audio file. Let's take a look.


However, `XLSR-Wav2Vec2` expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.

 Thankfully, `datasets` does this automatically by calling the other column `audio`. Let try it out. 

Great, we can see that the audio file has automatically been loaded. This is thanks to the new [`"Audio"` feature](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=audio#datasets.Audio) introduced in `datasets == 4.13.3`, which loads and resamples audio files on-the-fly upon calling.

In the example above we can see that the audio data is loaded with a sampling rate of 48kHZ whereas 16kHz are expected by the model. We can set the audio feature to the correct sampling rate by making use of [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column):

Let's take a look at `"audio"` again.

This seemed to have worked! Let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded. 

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [ ]:
# import IPython.display as ipd
# import numpy as np
# import random

# rand_int = random.randint(0, len(valid_dataset)-1)
# print(valid_dataset[rand_int]["audio"]["array"])
# ipd.Audio(data=valid_dataset[rand_int]["audio"]["array"], autoplay=True, rate=16000)

Great, it seems like the data is now correctly loaded and resampled. 

It can be heard, that the speakers change along with their speaking rate, accent, and background environment, etc. Overall, the recordings sound acceptably clear though, which is to be expected from a crowd-sourced read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [ ]:
rand_int = random.randint(0, len(test_dataset)-1)

print("Target text:", test_dataset[rand_int]["sentence"])
print("Input array shape:", test_dataset[rand_int]["audio"]["array"].shape)
print("Sampling rate:", test_dataset[rand_int]["audio"]["sampling_rate"])

Target text: و فن الرسم من الفنون المهمه جدا 
Input array shape: (38912,)
Sampling rate: 16000


Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can leverage `Wav2Vec2Processor` to process the data to the format expected by `Wav2Vec2ForCTC` for training. We will again make use of the `map(...)` function.

First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_values` from the loaded audio file. In our case, this includes only normalization, but for other speech models, this step could correspond to extracting, *e.g.* [Log-Mel features](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum). 
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [ ]:
def prepare_dataset(batch):
    
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"],return_tensors="pt",sampling_rate=audio["sampling_rate"],padding=True).input_values[0].to("cuda")
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"],return_tensors="pt",padding=True).input_ids.to('cuda')
    return batch

In [ ]:
def prepare_dataset2(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"],sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

Let's apply the data preparation function to all examples.

In [ ]:
train_dataset = train_dataset.map(prepare_dataset2, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset2, remove_columns=test_dataset.column_names)
#valid_dataset =valid_dataset.map(prepare_dataset2, remove_columns=valid_dataset.column_names)


  0%|          | 0/2901 [00:00<?, ?ex/s]

  0%|          | 0/153 [00:00<?, ?ex/s]

In [ ]:
test_dataset

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 153
})

**Note**: Currently `datasets` make use of [`torchaudio`](https://pytorch.org/audio/stable/index.html) and [`librosa`](https://librosa.org/doc/latest/index.html) for audio loading and resampling. If you wish to implement your own costumized data loading/sampling, feel free to just make use of the `"path"` column instead and disregard the `"audio"` column.

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
import transformers
print(transformers.__version__)

4.18.0


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor)

In [ ]:
data_collator=data_collator

Next, the evaluation metric is defined. As mentioned earlier, the 
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
from datasets import load_dataset, load_metric, Audio
wer_metric = load_metric("wer")


The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `XLSR-Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

Because the dataset is quite small (~6h of training data) and because Common Voice is quite noisy, fine-tuning Facebook's [wav2vec2-large-xlsr-53 checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) seems to require some hyper-parameter tuning. Therefore, I had to play around a bit with different values for dropout, [SpecAugment](https://arxiv.org/abs/1904.08779)'s masking dropout rate, layer dropout, and the learning rate until training seemed to be stable enough. 

**Note**: When using this notebook to train XLSR-Wav2Vec2 on another language of Common Voice those hyper-parameter settings might not work very well. Feel free to adapt those depending on your use case. 

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'project_q.weight', 'project_q.bias', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'quantizer.weight_proj.weight', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
train_dataset

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 2901
})

The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.8/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1674: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In addition, let's enable gradient checkpointing to save some memory.

In [ ]:
model.gradient_checkpointing_enable()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir="./22new_wav2vec2-large-xlsr-egyptian-demo",
  group_by_length=True,
  per_device_train_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=110,
  fp16=True,
  save_steps=100,
  eval_steps=500,
  logging_steps=10,
  learning_rate=5e-5,
  warmup_steps=1000,
  save_total_limit=5,
 
 
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
    #resume_from_checkpoint='/content/drive/MyDrive/Final_speech/ibrahimnew_wav2vec2-large-xlsr-egyptian-demo/checkpoint-6500'
)

Using amp half precision backend




---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take multiple hours depending on the GPU allocated to this notebook. While the trained model yields somewhat satisfying results on *Common Voice*'s test data of Turkish, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how XLSR-Wav2Vec2's [checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) can be fine-tuned on a low-resource ASR dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

The training and validation loss go down nicely. The WER starts improving only at a later stage. Because this notebook is just for demonstration purposes, we can stop here.

To fine-tune larger models on larger datasets using CTC loss, one should take a look at the official speech-recognition examples [here](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 🤗.

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.train('/content/drive/MyDrive/Final_speech/gabrnew_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14600')

Loading model from /content/drive/MyDrive/Final_speech/gabrnew_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14600).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2901
  Num Epochs = 110
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 2
  Total optimization steps = 19910
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 80
  Continuing training from global step 14600
  Will skip the first 80 epochs then the first 240 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command

  0%|          | 0/240 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss,Wer
15000,1.043300,0.501492,0.555138
15500,0.527500,0.244770,0.342105
16000,0.429700,0.234912,0.323308
16500,0.486400,0.242690,0.339599
17000,0.479400,0.255191,0.333333
17500,0.440200,0.247482,0.333333
18000,0.439200,0.279992,0.344612


Saving model checkpoint to ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14700
Configuration saved in ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14700/config.json
Model weights saved in ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14700/pytorch_model.bin
Feature extractor saved in ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14700/preprocessor_config.json
Saving model checkpoint to ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14800
Configuration saved in ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14800/config.json
Model weights saved in ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14800/pytorch_model.bin
Feature extractor saved in ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14800/preprocessor_config.json
Saving model checkpoint to ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14900
Configuration saved in ./22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14900/config.json
Model weights saved in ./22new_wav2vec

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-63-afcfc2344a2a>", line 1, in <module>
    trainer.train('/content/drive/MyDrive/Final_speech/gabrnew_wav2vec2-large-xlsr-egyptian-demo/checkpoint-14600')
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 1422, in train
    tr_loss_step = self.training_step(model, inputs)
  File "/usr/local/lib/python3.8/dist-packages/transformers/trainer.py", line 2021, in training_step
    self.scaler.scale(loss).backward()
  File "/usr/local/lib/python3.8/dist-packages/torch/_tensor.py", line 488, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.8/dist-packages/torch/autograd/__init__.py", line 197, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During ha

KeyboardInterrupt: ignored

In [ ]:
trainer.predict(test_dataset)

In [ ]:
!cp -r '/content/22new_wav2vec2-large-xlsr-egyptian-demo' '/content/drive/MyDrive/Final_speech/' 

In [ ]:
#!kill $(ps aux | awk '{print $2}')

##Run until here 

In [ ]:
trainer.save_model("container_0/wav2vec2-large-xlsr-Egyptian")

In [ ]:
%%capture
!pip install datasets==2.1.0
!pip install transformers==4.18.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

In [ ]:
import pandas as pd
dftest = pd.DataFrame(columns=['audio'])
from datasets import Dataset
dftest['audio']=['/content/9266.wav']  ## audio path 
test_dataset1 = Dataset.from_pandas(dftest)

In [ ]:
dftest.head()

In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# test_dataset = #TODO: WRITE YOUR CODE TO LOAD THE TEST DATASET. For sample see the Colab link in Training Section.
model = Wav2Vec2ForCTC.from_pretrained("/content/22new_wav2vec2-large-xlsr-egyptian-demo/checkpoint-17900")
processor = Wav2Vec2Processor.from_pretrained("/content/drive/MyDrive/Final_speech/gabrprocessor ")

resampler = torchaudio.transforms.Resample(44100, 16_000) # The original data was with 48,000 sampling rate. You can change it according to your input.

# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["audio"])
    batch["audio"] = resampler(speech_array).squeeze().numpy()
    return batch


#print("Reference:",train_test['test']["sentence"][0:5])

In [ ]:
test_dataset = test_dataset1.map(speech_file_to_array_fn)
inputs = processor(test_dataset["audio"], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
#print("ground truth :",test_dataset['sentence'][1:10])


In [ ]:
###
dftest=['audio']

In [ ]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

# # test_dataset = #TODO: WRITE YOUR CODE TO LOAD THE TEST DATASET. For sample see the Colab link in Training Section.
# test_dataset = dataset['test'].map(remove_special_characters)

wer = load_metric("wer")

# processor = Wav2Vec2Processor.from_pretrained("container_0/wav2vec2-large-xlsr-kn")
# model = Wav2Vec2ForCTC.from_pretrained("container_0/wav2vec2-large-xlsr-kn")
model.to("cuda")

# chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\–\…]'
# resampler = torchaudio.transforms.Resample(48_000, 16_000)

# # Preprocessing the datasets.
# # We need to read the aduio files as arrays
# def speech_file_to_array_fn(batch):
#     batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
#     speech_array, sampling_rate = torchaudio.load(batch["path"])
#     batch["speech"] = resampler(speech_array).squeeze().numpy()
#     return batch

# test_dataset = test_dataset.map(speech_file_to_array_fn)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
    inputs = processor(batch["audio"], sampling_rate=16_000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits
        pred_ids = torch.argmax(logits, dim=-1)
        batch["pred_strings"] = processor.batch_decode(pred_ids)
        return batch

result = test_dataset1.map(evaluate, batched=True, batch_size=8)
# NOTE : The original test set was different and this result may be skewed. The original test set gave around 14.53% WER.
print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))